In [11]:
import pandas as pd
import plotly.plotly as py
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
from collections import Counter


print('Loaded Libraries')

Loaded Libraries


In [34]:
train_data = pd.read_csv("cleaned_data.csv", encoding = "ISO-8859-1")

train_data.drop('nkill', 1, inplace=True)   
train_data.drop('nwound', 1, inplace=True)
train_data.drop('propvalue', 1, inplace=True)
train_data.drop('iyear', 1, inplace=True)

train_data

,success,nperps,suicide,total_num_hurt_killed,Australasia & Oceania,Central America & Caribbean,Central Asia,East Asia,Eastern Europe,Middle East & North Africa,...,Catastrophic (likely > $1 billion),Major (likely > $1 million but < $1 billion),Minor (likely < $1 million),Chemical,Explosives/Bombs/Dynamite,Firearms,Incendiary,Melee,Sabotage Equipment,"Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)"
0,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,1,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,1,3,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,1,3,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
5,1,4,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
6,1,3,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
7,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
8,1,4,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
9,1,42,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [37]:
from sklearn.model_selection import train_test_split
import numpy as np



train_data

# Split into training and validation sets
train, validation = train_test_split(train_data, 
                                        shuffle = True,
                                        test_size=0.2,
                                        random_state=42
                                        )


# Split data into input and outputs
X_train = train.loc[:, train.columns != 'total_num_hurt_killed']
y_train = train['total_num_hurt_killed']

X_val = validation.loc[:, validation.columns != 'total_num_hurt_killed']
y_val = validation['total_num_hurt_killed']

X_val.head()

,success,nperps,suicide,Australasia & Oceania,Central America & Caribbean,Central Asia,East Asia,Eastern Europe,Middle East & North Africa,North America,...,Catastrophic (likely > $1 billion),Major (likely > $1 million but < $1 billion),Minor (likely < $1 million),Chemical,Explosives/Bombs/Dynamite,Firearms,Incendiary,Melee,Sabotage Equipment,"Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)"
3944,1,15,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2691,1,2,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
227,1,5,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1261,1,50,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2665,1,16,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [98]:
from matplotlib import pyplot as plt
import math
from math import sqrt
from sklearn.metrics import mean_squared_error
# from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression



# train_predictions

# mean_squared_error(train_predictions, all_zero)
# np.sum(pd.Series.abs(train_predictions - all_zero))

classifiers = [
    DecisionTreeRegressor(),
    GradientBoostingRegressor(),
    RandomForestRegressor(),
    AdaBoostRegressor(),
    KNeighborsRegressor(5),
    LinearRegression()
    ]

# Logging for Visual Comparison
log_cols=["Regressor", "MSE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_val)
    print(train_predictions)
    mse = sqrt(mean_squared_error(y_val, train_predictions))
    print("MSE: {}".format(mse))
    
    log_entry = pd.DataFrame([[name, mse]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

DecisionTreeRegressor
****Results****
[0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.28000000e+02 1.55555556e+00 6.38888889e+01
 1.00000000e+00 6.00000000e+00 4.83333333e+01 2.00000000e+00
 0.00000000e+00 0.00000000e+00 1.05000000e+01 2.50000000e+00
 1.33333333e+00 2.00000000e+00 5.71428571e-02 4.00000000e+00
 1.00000000e+00 1.34000000e+02 6.24736842e+01 2.60000000e+00
 5.71428571e-02 0.00000000e+00 0.00000000e+00 1.33333333e+00
 1.10000000e+01 7.00000000e+00 1.00000000e+00 5.71428571e-02
 5.77777778e+01 1.00000000e+00 2.00000000e+00 2.23809524e+00
 3.00000000e+00 1.62439024e+01 0.00000000e+00 2.00000000e+00
 0.00000000e+00 1.00000000e-01 2.93000000e+01 1.25000000e+00
 2.11111111e+00 4.00000000e+00 5.71428571e-02 6.00000000e+00
 5.77777778e+01 6.00000000e+00 0.00000000e+00 2.45714286e+01
 0.00000000e+00 2.55862069e+01 1.60000000e+00 1.23333333e+01
 2.65000000e+01 2.57500000e+01 3.00000000e+00 1.00000000e+00
 0.00000000e+00 8.00000000e+00 6.24736842e+01 3

In [96]:
all_zero = len(y_val) * [0]
print(sqrt(mean_squared_error(all_zero, y_val)))

33.47770707554418
